In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from elopy.elo import Elo
import datetime
import time

# Read recipe inputs
# Dataset nfl_scores_5 renamed to nfl_scores_post_Divional_Rd by jjorgenson on 2023-02-01 16:31:49
spreadspoke_scores = dataiku.Dataset("nfl_scores_post_Divional_Rd")
games_df = spreadspoke_scores.get_dataframe()

In [0]:
games_df.head()

In [17]:
#commenting out formula that was in place to use variables for days back
#days_back = int(dataiku.get_custom_variables()["days_back"])
#completely commeting out days back - not used in new Start function
#ays_back = 90

#Commenting out Start parameters - trying to find a custom start date
#tart = datetime.datetime.now() - datetime.timedelta(days=days_back)
#new start is where the Elo model should start rating things -- formerly was Now minus X days setup as a variable
#string = "01/08/2022"
#start_string = datetime.datetime.strptime(string,"%d/%m/%Y")
#datetime format = yyyy,month,day
start = datetime.datetime(2022,8,1)
#start = datetime.datetime.timestamp(start_string)
#print(start)

#commenting out query to look for schedule dates that are after Start AND have blank score -- skip looking for blank score
#qry = f'schedule_date>\'{start}+00:00\' & score_home.notna()'
qry = f'schedule_date>\'{start}+00:00\' & score_home.notna()'
hist_games_df = games_df.query(qry, engine='python')

2022-08-01 00:00:00


In [15]:
len(hist_games_df)

281

In [16]:
teams = {}
for team in hist_games_df['team_home'].unique():
    teams[team] = Elo(start_elo=1600, k=20, hca = 50)

for idx, row in hist_games_df.sort_values(['schedule_date']).iterrows():
    if row['score_home'] > row['score_away']:
        winby = row['score_home'] - row['score_away']
        teams[row['team_home']].play_game(teams[row['team_away']], winby, is_home=True)
    else:
        winby = row['score_away'] - row['score_home']
        teams[row['team_away']].play_game(teams[row['team_home']], winby, is_home=False)

#     sprd = 0-teams[row['team_home']].point_spread(teams[row['team_away']], is_home=True)

    hist_games_df.at[idx, 'home_elo'] = teams[row['team_home']].elo
    hist_games_df.at[idx, 'away_elo'] = teams[row['team_away']].elo
#     hist_games_df.at[idx, 'pred_spread'] = sprd

/data/dataiku/dss_data/code-envs/python/elo/lib64/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/data/dataiku/dss_data/code-envs/python/elo/lib64/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## Record the team ELO Ratings

In [0]:
results = []
for team in hist_games_df['team_home'].unique():
    print(team, teams[team].elo)
    results.append({'team': team, 'rating': teams[team].elo})

## Grab the upcoming games

In [0]:
days_ahead = int(dataiku.get_custom_variables()["days_ahead"])

start = datetime.datetime.now() + datetime.timedelta(days=days_ahead)
qry = f'schedule_date<\'{start}+00:00\' & not score_home.notna()'
upcoming_games_df = games_df.query(qry, engine='python')

In [0]:
# upcoming_games_df.head()

## Computing the odds of a home team win

In [0]:
tw_pred = []
for idx, row in upcoming_games_df.iterrows():
    tw_pred.append({'home_team':row['team_home'],
                    'away_team':row['team_away'],
                    'date': row['schedule_date'].strftime("%Y-%m-%d"),
                    'spread': 0-(teams[row['team_home']].point_spread(teams[row['team_away']], is_home=True)),
                    'home_odds':teams[row['team_home']].win_probs(teams[row['team_away']], is_home=True)})

In [0]:
tw_pred

In [0]:
# Write ELO outputs
# Dataset elo_results_5 renamed to elo_results_post_Divisional_Rd by jjorgenson on 2023-02-01 16:30:32
elo_results = dataiku.Dataset("elo_results_post_Divisional_Rd")
elo_results.write_with_schema(pd.DataFrame.from_dict(results))

In [0]:
# Dataset upcoming_game_probabilities_5 renamed to upcoming_game_probabilities_Conf_Champ_Wk by jjorgenson on 2023-02-01 16:30:56
upcoming_results = dataiku.Dataset("upcoming_game_probabilities_Conf_Champ_Wk")
upcoming_results.write_with_schema(pd.DataFrame.from_dict(tw_pred))